<a href="https://colab.research.google.com/github/Ki6an/Machine_Learning_Projects/blob/master/Music%20Generation/Colab/Music_Melodies_Gen_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from tensorflow import keras
from music21 import converter, stream, key, interval, note
import music21
import os
import json
import numpy as np
 
 
KERN_DATASET_PATH = '/content/drive/My Drive/Machine Learning /data/erk'
SAVE_DIR = "dataset"
SINGLE_FILE_DATASET = "file_dataset"
MAPPING_PATH = "mapping.json"
SEQUENCE_LENGTH = 64
 
ACCEPTABLE_DURATIONS = [
    0.25,
    0.5,
    0.75,
    1.0,
    1.5,
    2,
    3,
    4
]
 
 
def has_acceptable_duration(song, acceptable_durations):
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            return False
    return True
 
 
def load_songs_in_kern(dataset_path):
    pass
    songs = []
 
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:
 
            # consider only kern files
            if file[-3:] == 'krn':
                song = converter.parse(os.path.join(path, file))
                songs.append(song)
    return songs
 
 
def transpose(song):
    # get the key from the song
    parts = song.getElementsByClass(stream.Part)
    measures_part0 = parts[0].getElementsByClass(stream.Measure)
    key_x = measures_part0[0][4]
 
    # estimate key using music21
    if not isinstance(key_x, key.Key):
        key_x = song.analyze('key')
 
    print(key_x)
 
    # get interval for transposition, Eg: Bmaj to Cmaj
    if key_x.mode == 'major':
        interval_x = music21.interval.Interval(key_x.tonic, music21.pitch.Pitch('C'))
    elif key_x.mode == 'minor':
        interval_x = music21.interval.Interval(key_x.tonic, music21.pitch.Pitch('A'))
 
    # transpose song by calculated interval
    transposed_song = song.transpose(interval_x)
 
    return transposed_song
 
 
def encode_song(song, time_step=0.25):
    """Converts a score into a time-series-like music representation. Each item in the encoded list represents 'min_duration'
    quarter lengths. The symbols used at each step are: integers for MIDI notes, 'r' for representing a rest, and '_'
    for representing notes/rests that are carried over into a new time step. Here's a sample encoding:
        ["r", "_", "60", "_", "_", "_", "72" "_"]
    :param song (m21 stream): Piece to encode
    :param time_step (float): Duration of each time step in quarter length
    :return:
    """
 
    encoded_song = []
 
    for event in song.flat.notesAndRests:
 
        # handle notes
        if isinstance(event, note.Note):
            symbol = event.pitch.midi # 60
        # handle rests
        elif isinstance(event, note.Rest):
            symbol = "r"
 
        # convert the note/rest into time series notation
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):
 
            # if it's the first time we see a note/rest, let's encode it. Otherwise, it means we're carrying the same
            # symbol in a new time step
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")
 
    # cast encoded song to str
    encoded_song = " ".join(map(str, encoded_song))
 
    return encoded_song
 
 
def pre_process(dataset_path):
 
    # load folk songs
    print("Loading songs...")
    songs = load_songs_in_kern(dataset_path)
    print(f"Loaded {len(songs)} songs.")
 
    for i, song in enumerate(songs):
 
        # filter out songs that have non-acceptable durations
        if not has_acceptable_duration(song, ACCEPTABLE_DURATIONS):
            continue
 
        # transpose songs to Cmaj/Amin
        song = transpose(song)
 
        # encode songs with music time series representation
        encoded_song = encode_song(song)
 
        # save songs to text file
        save_path = os.path.join(SAVE_DIR, str(i))
        with open(save_path, "w") as fp:
            fp.write(encoded_song)
 
 
def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song
 
 
def create_single_file_dataset(dataset_path, file_dataset_path, sequence_length):
    """Generates a file collating all the encoded songs and adding new piece delimiters.
    :param dataset_path (str): Path to folder containing the encoded songs
    :param file_dataset_path (str): Path to file for saving songs in single file
    :param sequence_length (int): # of time steps to be considered for training
    :return songs (str): String containing all songs in dataset + delimiters
    """
 
    new_song_delimiter = "/ " * sequence_length
    songs = ""
 
    # load encoded songs and add delimiters
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            songs = songs + song + " " + new_song_delimiter
 
    # remove empty space from last character of string
    songs = songs[:-1]
 
    # save string that contains all the dataset
    with open(file_dataset_path, "w") as fp:
        fp.write(songs)
 
    return songs
 
 
def create_mapping(songs, mapping_path):
    """Creates a json file that maps the symbols in the song dataset onto integers
    :param songs (str): String with all songs
    :param mapping_path (str): Path where to save mapping
    :return:
    """
    mappings = {}
 
    # identify the vocabulary
    songs = songs.split()
    vocabulary = list(set(songs))
 
    # create mappings
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i
 
    # save voabulary to a json file
    with open(mapping_path, "w") as fp:
        json.dump(mappings, fp, indent=4)
 
def convert_songs_to_int(songs):
    int_songs = []
 
    # load mappings
    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)
 
    # transform songs string to list
    songs = songs.split()
 
    # map songs to int
    for symbol in songs:
        int_songs.append(mappings[symbol])
 
    return int_songs
 
 
def generate_training_sequences(sequence_length):
    """Create input and output data samples for training. Each sample is a sequence.
    :param sequence_length (int): Length of each sequence. With a quantisation at 16th notes, 64 notes equates to 4 bars
    :return inputs (ndarray): Training inputs
    :return targets (ndarray): Training targets
    """
 
    # load songs and map them to int
    songs = load(SINGLE_FILE_DATASET)
    int_songs = convert_songs_to_int(songs)
 
    inputs = []
    targets = []
 
    # generate the training sequences
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])
 
    # one-hot encode the sequences
    vocabulary_size = len(set(int_songs))
    # inputs size: (# of sequences, sequence length, vocabulary size)
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
    targets = np.array(targets)
 
    return inputs, targets
 
 
def main():
    pre_process(KERN_DATASET_PATH)
    songs = create_single_file_dataset(SAVE_DIR, SINGLE_FILE_DATASET, SEQUENCE_LENGTH)
    create_mapping(songs, MAPPING_PATH)
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)
    # for debugging   a = 1
 
 
main()

In [3]:
import tensorflow.keras as keras

OUTPUT_UNITS = 38
NUM_UNITS = [256]
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 64
SAVE_MODEL_PATH = "model.h5"


def build_model(output_units, num_units, loss, learning_rate):
    """Builds and compiles model
    :param output_units (int): Num output units
    :param num_units (list of int): Num of units in hidden layers
    :param loss (str): Type of loss function to use
    :param learning_rate (float): Learning rate to apply
    :return model (tf model): Where the magic happens :D
    """

    # create the model architecture
    input = keras.layers.Input(shape=(None, output_units))
    x = keras.layers.LSTM(num_units[0])(input)
    x = keras.layers.Dropout(0.2)(x)

    output = keras.layers.Dense(output_units, activation="softmax")(x)

    model = keras.Model(input, output)

    # compile model
    model.compile(loss=loss,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=["accuracy"])

    model.summary()

    return model


def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    """Train and save TF model.
    :param output_units (int): Num output units
    :param num_units (list of int): Num of units in hidden layers
    :param loss (str): Type of loss function to use
    :param learning_rate (float): Learning rate to apply
    """

    # generate the training sequences
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)

    # build the network
    model = build_model(output_units, num_units, loss, learning_rate)

    # train the model
    model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)

    # save the model
    model.save(SAVE_MODEL_PATH)

train()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 38)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               302080    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 38)                9766      
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
5660/5660 [==============================] - 112s 20ms/step - loss: 0.6922 - accuracy: 0.7870
Epoch 2/50
5660/5660 [==============================] - 112s 20ms/step - loss: 0.5788 - accuracy: 0.8130
Epoch 3/50
5660/5660 [==============

In [10]:
import json
import numpy as np
import tensorflow.keras as keras
import music21 as m21
# from preprocess import SEQUENCE_LENGTH, MAPPING_PATH

class MelodyGenerator:
    """A class that wraps the LSTM model and offers utilities to generate melodies."""

    def __init__(self, model_path="model.h5"):
        """Constructor that initialises TensorFlow model"""

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH


    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        """Generates a melody using the DL model and returns a midi file.
        :param seed (str): Melody seed with the notation used to encode the dataset
        :param num_steps (int): Number of steps to be generated
        :param max_sequence_len (int): Max number of steps in seed to be considered for generation
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.
        :return melody (list of str): List with symbols representing a melody
        """

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # (1, max_sequence_length, num of symbols in the vocabulary)
            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self._sample_with_temperature(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def _sample_with_temperature(self, probabilites, temperature):
        """Samples an index from a probability array reapplying softmax using temperature
        :param predictions (nd.array): Array containing probabilities for each of the possible outputs.
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.
        :return index (int): Selected output symbol
        """
        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index


    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):
        """Converts a melody into a MIDI file
        :param melody (list of str):
        :param min_duration (float): Duration of each time step in quarter length
        :param file_name (str): Name of midi file
        :return:
        """

        # create a music21 stream
        stream = m21.stream.Stream()

        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1

        # write the m21 stream to a midi file
        stream.write(format, file_name)



mg = MelodyGenerator()
seed = "67 _ 67 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
seed2 = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
melody = mg.generate_melody(seed, 500, SEQUENCE_LENGTH, 0.3)
print(melody)
mg.save_melody(melody)



['67', '_', '67', '_', '67', '_', '_', '65', '64', '_', '64', '_', '64', '_', '_', '_', '67', '_', '_', '_', '64', '_', '_', '_', '62', '_', '_', '_', '60', '_', '_', '_', 'r', '_', '_', '_', '67', '_', '_', '_', '64', '_', '_', '_', '67', '_', '_', '_', '69', '_', '67', '_', '65', '_', '64', '_', '65', '_', '_', '_', '65', '_', '_', '_', '67', '_', '65', '_', '64', '_', '_', '_', '64', '_', '_', '_', '67', '_', '64', '_', '64', '_', '62', '_', '62', '_', '_', '_', '69', '_', '_', '_', '67', '_', '_', '_', '64', '_', '_', '_', '62', '_', '_', '_', '60', '_', '_', '_', '_', '_', '_', '_']
